# Linear codes for Inner Product Masking

### $n=2$ shares, $\ell=8$ bits

Parameters:

- $Z=(X + L_2Y_2, Y_2)$ where $X, Y$ and $Z$ are the sensitive variable, a mask and the protected variable, respectively
- $L_2\in \mathbb{F}_{2^\ell}\backslash\{0\}$, thus there are 255 linear codes for IPM
- Each element over $\mathbb{F}_{2^\ell}$ can be denoted as $\alpha^i$ where $i\in\{0, 1, \ldots, 254\}$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd # Pandas for tables
from IPython.display import Latex
from IPython.display import HTML

In [2]:
def read_f(file_name):
    """Reading weight enumerators."""
    with open(file_name, 'r') as fp:
        wd = fp.read().split("]\n")[:-1] # "\n"
        wd = np.array([list(map(int, re.findall(r"\d+", elem))) for elem in wd])
        
    return wd

## 1. Loading all weight enumerators

In [3]:
wd = read_f("weight_distrib_n2k8.txt") # Weight distribution

# print(wd.shape) # 256 entries: 255 for IPM codes and one for BKLC codes

### 1.1 Generating values

In [4]:
alpha_all = ['$\\alpha^{%d}$'%i for i in np.arange(len(wd))]
d_all = np.zeros(len(wd))
B_all = np.zeros(len(wd))
for i in range(len(wd)):
    d_all[i] = wd[i][2]
    B_all[i] = wd[i][3]

### 1.2 Defining styles of dataframe

We borrow the following setting of dataframe from https://mode.com/example-gallery/python_dataframe_styling/

In [5]:
# Set CSS properties for th, td and caption elements in dataframe
th_props = [('font-size', '14px'), ('text-align', 'left'), ('font-weight', 'bold'), ('background-color', '#E0E0E0')]
td_props = [('font-size', '13px'), ('text-align', 'left'), ('min-width', '80px')]
cp_props = [('font-size', '16px'), ('text-align', 'center')]
# Set table styles
styles = [dict(selector="th", props=th_props), dict(selector="td", props=td_props), dict(selector="caption", props=cp_props)]
cm_1 = sns.light_palette("red", as_cmap=True)
cm_2 = sns.light_palette("purple", as_cmap=True, reverse=True)

In [6]:
df = pd.DataFrame({'$L_2$': alpha_all[:-2], '$d_{\mathcal{D}}^\perp$': d_all[:-2], '$B_{d_{\mathcal{D}}^\perp}$': B_all[:-2], 'Weight Enumerators': wd[:-2]})

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.width', 800)
(df.style
    .background_gradient(cmap=cm_1, subset=['$d_{\mathcal{D}}^\perp$','$B_{d_{\mathcal{D}}^\perp}$' ])
    .background_gradient(cmap=cm_2, subset=['$B_{d_{\mathcal{D}}^\perp}$' ])
    .set_caption('Tab. I All linear codes for IPM with $n=2$ shares over $\mathbb{F}_{2^8}$.')
    .set_table_styles(styles))


,$L_2$,$d_{\mathcal{D}}^\perp$,$B_{d_{\mathcal{D}}^\perp}$,Weight Enumerators
0,$\alpha^{0}$,2,8,"[0, 1, 2, 8, 4, 28, 6, 56, 8, 70, 10, 56, 12, 28, 14, 8, 16, 1]"
1,$\alpha^{1}$,2,7,"[0, 1, 2, 7, 4, 21, 5, 8, 6, 35, 7, 32, 8, 35, 9, 48, 10, 21, 11, 32, 12, 7, 13, 8, 14, 1]"
2,$\alpha^{2}$,2,6,"[0, 1, 2, 6, 4, 15, 5, 16, 6, 24, 7, 48, 8, 31, 9, 48, 10, 30, 11, 16, 12, 17, 14, 4]"
3,$\alpha^{3}$,2,5,"[0, 1, 2, 5, 4, 10, 5, 20, 6, 24, 7, 48, 8, 41, 9, 40, 10, 33, 11, 16, 12, 12, 13, 4, 14, 2]"
4,$\alpha^{4}$,2,4,"[0, 1, 2, 4, 4, 6, 5, 24, 6, 24, 7, 44, 8, 53, 9, 36, 10, 28, 11, 20, 12, 12, 13, 4]"
5,$\alpha^{5}$,2,3,"[0, 1, 2, 3, 4, 3, 5, 24, 6, 29, 7, 38, 8, 57, 9, 46, 10, 23, 11, 18, 12, 11, 13, 2, 14, 1]"
6,$\alpha^{6}$,2,2,"[0, 1, 2, 2, 4, 1, 5, 23, 6, 32, 7, 40, 8, 55, 9, 46, 10, 30, 11, 16, 12, 7, 13, 3]"
7,$\alpha^{7}$,2,1,"[0, 1, 2, 1, 4, 1, 5, 23, 6, 36, 7, 40, 8, 51, 9, 46, 10, 33, 11, 16, 12, 3, 13, 3, 14, 2]"
8,$\alpha^{8}$,4,3,"[0, 1, 4, 3, 5, 25, 6, 34, 7, 36, 8, 55, 9, 46, 10, 28, 11, 20, 12, 5, 13, 1, 14, 2]"
9,$\alpha^{9}$,4,5,"[0, 1, 4, 5, 5, 23, 6, 29, 7, 40, 8, 59, 9, 46, 10, 26, 11, 16, 12, 7, 13, 3, 14, 1]"


## 2. Optimal linear codes for IPM

### 2.1 Linear codes with $d_{\mathcal{D}}^\perp=4$

We focus on the the linear codes with greater $d_{\mathcal{D}}^\perp$, which are better in the sense of side-channel resistance (from our paper).

In [7]:
# Finding the indices of d_C=4 
d_index = []
d_C = 4
for i in range(len(wd)):
    if wd[i][2] == d_C:
        d_index.append(i)

#d_index = np.array(d_index)

In [8]:
def highlight(s, threshold, column):
    is_min = pd.Series(data=False, index=s.index)
    is_min[column] = (s.loc[column] <= threshold)
    return ['background-color: gold' if is_min.any() else '' for v in is_min]

In [9]:
df_4 = pd.DataFrame({'$L_2$': np.array(alpha_all)[d_index], '$d_{\mathcal{D}}^\perp$': d_all[d_index], '$B_{d_{\mathcal{D}}^\perp}$': B_all[d_index], 'Weight Enumerators': wd[d_index]})
df_4 = df_4.sort_values(by=['$B_{d_{\mathcal{D}}^\perp}$'], ascending=True)

(df_4.style
    .apply(highlight, threshold=3, column=['$B_{d_{\mathcal{D}}^\perp}$'], axis=1)
    .background_gradient(cmap=cm_2, subset=['$B_{d_{\mathcal{D}}^\perp}$' ])
    .set_caption('Tab. II Linear codes for IPM with $d_{\mathcal{D}}^\perp=4$.')
    .set_table_styles(styles))

,$L_2$,$d_{\mathcal{D}}^\perp$,$B_{d_{\mathcal{D}}^\perp}$,Weight Enumerators
0,$\alpha^{8}$,4,3,"[0, 1, 4, 3, 5, 25, 6, 34, 7, 36, 8, 55, 9, 46, 10, 28, 11, 20, 12, 5, 13, 1, 14, 2]"
48,$\alpha^{129}$,4,3,"[0, 1, 4, 3, 5, 25, 6, 33, 7, 36, 8, 59, 9, 46, 10, 22, 11, 20, 12, 9, 13, 1, 14, 1]"
47,$\alpha^{128}$,4,3,"[0, 1, 4, 3, 5, 25, 6, 34, 7, 36, 8, 55, 9, 46, 10, 28, 11, 20, 12, 5, 13, 1, 14, 2]"
45,$\alpha^{126}$,4,3,"[0, 1, 4, 3, 5, 25, 6, 33, 7, 36, 8, 59, 9, 46, 10, 22, 11, 20, 12, 9, 13, 1, 14, 1]"
46,$\alpha^{127}$,4,3,"[0, 1, 4, 3, 5, 25, 6, 34, 7, 36, 8, 55, 9, 46, 10, 28, 11, 20, 12, 5, 13, 1, 14, 2]"
93,$\alpha^{247}$,4,3,"[0, 1, 4, 3, 5, 25, 6, 34, 7, 36, 8, 55, 9, 46, 10, 28, 11, 20, 12, 5, 13, 1, 14, 2]"
82,$\alpha^{214}$,4,4,"[0, 1, 4, 4, 5, 22, 6, 36, 7, 44, 8, 45, 9, 40, 10, 36, 11, 20, 12, 6, 13, 2]"
81,$\alpha^{213}$,4,4,"[0, 1, 4, 4, 5, 21, 6, 36, 7, 44, 8, 45, 9, 46, 10, 36, 11, 12, 12, 6, 13, 5]"
44,$\alpha^{125}$,4,4,"[0, 1, 4, 4, 5, 23, 6, 31, 7, 42, 8, 59, 9, 40, 10, 24, 11, 22, 12, 8, 13, 1, 14, 1]"
12,$\alpha^{42}$,4,4,"[0, 1, 4, 4, 5, 21, 6, 36, 7, 44, 8, 45, 9, 46, 10, 36, 11, 12, 12, 6, 13, 5]"


### 2.2 Optimal codes for IPM

As shown in our paper, the codes satifying two conditions are optimal:

- Maximizing $d_{\mathcal{D}}^\perp$, here the $\max\{d_{\mathcal{D}}^\perp\} = 4$
- Minimizing $B_{d_{\mathcal{D}}^\perp}$, here the $\min\{B_{d_{\mathcal{D}}^\perp}\} = 3$

Note that we use two leakage detection metrics **SNR** (signal-to-noise ratio) and **MI** (mutual information), and one leakage exploitation metric **SR** (success rate) to assess the side-channel resistance of IPM with different codes.

As a result of Tab. II, we conclude that the optimal codes for IPM are genetated by $\mathbf{H}=[L_2, 1]$ where $L_2\in\{\alpha^8, \alpha^{126}, \alpha^{127}, \alpha^{128}, \alpha^{129}, \alpha^{247}\}$.

### 2.3 BKLC code with parameter $[16, 8, 5]$

BKLC is the short of Best Known Linear Code. Note that the code $[16, 8, 5]$ is unique.

In [10]:
bklc_index = [255]
cm_3 = sns.light_palette("red", as_cmap=True, reverse=True)
df_bklc = pd.DataFrame({'$L_2$': np.array(alpha_all)[-1], '$d_{\mathcal{D}}^\perp$': d_all[-1], '$B_{d_{\mathcal{D}}^\perp}$': B_all[-1], 'Weight Enumerators': wd[bklc_index]})

(df_bklc.style
    .background_gradient(cmap=cm_3, subset=['$d_{\mathcal{D}}^\perp$', '$B_{d_{\mathcal{D}}^\perp}$'])
    .set_caption('Tab. III A BKLC code for IPM with $d_{\mathcal{D}}^\perp=5$.')
    .set_table_styles(styles))

,$L_2$,$d_{\mathcal{D}}^\perp$,$B_{d_{\mathcal{D}}^\perp}$,Weight Enumerators
0,$\alpha^{255}$,5,24,"[0, 1, 5, 24, 6, 44, 7, 40, 8, 45, 9, 40, 10, 28, 11, 24, 12, 10]"


We can see that this BKLC code is better than all linear codes in IPM. It is interesting to notice that the BKLC code $[16, 8, 5]$ cannot be used in IPM, since it cannot be generated by $\mathbf{H}=[L_2, 1]$ with any $L_2\in\mathbb{F}_{2^8}$.